In [3]:
import os
from openslide import OpenSlide
from PIL import Image

def split_mrxs_to_tiles(input_path, output_dir, patch_size=1024, overlap=128, level=0):
    os.makedirs(output_dir, exist_ok=True)
    slide = OpenSlide(input_path)
    dims = slide.level_dimensions[level]
    
    for y in range(0, dims[1], patch_size - overlap):
        for x in range(0, dims[0], patch_size - overlap):
            width = min(patch_size, dims[0] - x)
            height = min(patch_size, dims[1] - y)
            
            tile = slide.read_region(
                location=(x, y),
                level=level,
                size=(width, height)
            )
            
            # Sprawdź, czy fragment jest całkowicie przezroczysty
            if tile.mode == 'RGBA':
                alpha = tile.getchannel('A')
                # Jeśli maksymalna wartość alfa wynosi 0, fragment jest przezroczysty
                if alpha.getextrema()[1] == 0:
                    continue  # Pomijamy ten fragment
                tile = tile.convert('RGB')
            
            tile.save(
                os.path.join(output_dir, f'tile_{x}_{y}.tif'),
                compression=None,
                tiffinfo={271: 'XYZ Scanner'}
            )
    
    slide.close()

In [4]:
split_mrxs_to_tiles(
    input_path='../data/Patomorfologia/1M01.mrxs',
    output_dir='../temp/1M01',
    patch_size=2048,  
    overlap=256,      
    level=0          
)